<a href="https://colab.research.google.com/github/graphitical/HeadsUP/blob/master/PDF_to_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get the right packages

In [ ]:
import sys, pathlib, os
!pip install pdf2image
!pip install poppler-utils
!apt-get install poppler-utils
from pdf2image import convert_from_path

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 0s (1,852 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


#Mount Google Drive

In [ ]:
# Use this to mount your google account that has
# access to the shared drive
import os, sys, datetime
from google.colab import drive
drive.mount('/gdrive')

# Change dir to shared drive
BASE_PATH = '/gdrive/Shareddrives/deep learning and PDF accessibility/'
os.chdir(BASE_PATH)
!pwd

Mounted at /gdrive
/gdrive/Shareddrives/deep learning and PDF accessibility


#Unpack PDFs

In [ ]:
DATA_PATH = 'dataset/CHI-2019'
DATA_SET = 'CHI-2019-pdfs-RETITLED.zip'
os.chdir(os.path.join(BASE_PATH,DATA_PATH))
if not os.path.exists(os.path.join(BASE_PATH,DATA_PATH,'CHI-2019-pdfs-RETITLED')):
  !unzip $DATA_SET
else:
  print('already unzipped')

Archive:  CHI-2019-pdfs-RETITLED.zip
   creating: CHI-2019-pdfs-RETITLED/
  inflating: CHI-2019-pdfs-RETITLED/MicrosoftWordpaper493docx.pdf  
  inflating: CHI-2019-pdfs-RETITLED/AutomatingtheAdministrationandAnalysisofPsychiatricTestsTheCaseofAttachmentinSchoolAgeChildren.pdf  
  inflating: CHI-2019-pdfs-RETITLED/AggregatedVisualizationofPlaytestingData.pdf  
  inflating: CHI-2019-pdfs-RETITLED/FeelingFireworksAnInclusiveTactileFireworkDisplay.pdf  
  inflating: CHI-2019-pdfs-RETITLED/ModelingMobileInterfaceTappabilityUsingCrowdsourcingandDeepLearning.pdf  
  inflating: CHI-2019-pdfs-RETITLED/InsertYourTitleHere.pdf  
  inflating: CHI-2019-pdfs-RETITLED/CrossDeviceTaxonomySurveyOpportunitiesandChallengesofInteractionsSpanningAcrossMultipleDevices.pdf  
  inflating: CHI-2019-pdfs-RETITLED/PseudoHapticWeightChangingthePerceivedWeightofVirtualObjectsByManipulatingControlDisplayRatio.pdf  
  inflating: CHI-2019-pdfs-RETITLED/DiagnosingandCopingwithModeErrorsinKoreanEnglishDuallanguageKeybo

#Convert to images

In [ ]:
import fnmatch
def pdf_to_images(input_dir, output_dir, img_type='jpeg'):
  fcount = 0
  skipcount = 0
  # make output dir
  if not os.path.exists(output_dir): os.mkdir(output_dir)
  
  pdf_list = pathlib.Path(input_dir).rglob('*.pdf')
  print('{} PDFs ready to be converted'.format(len(fnmatch.filter(os.listdir(input_dir), '*.pdf'))))

  for jj, pdf in enumerate(pdf_list):
    print('{}: Converting {}'.format(jj,pdf.name))
    try:
      pages = convert_from_path(pdf)
    except:
      print('PDF Read Error. Skipping',pdf.name)
      skipcount += 1
      continue

    for ii, page in enumerate(pages):
      file_path = os.path.join(output_dir,pdf.stem+'-{}.{}'.format(ii,img_type))
      if os.path.exists(file_path): 
        print('{} already converted. Skipping.'.format(pdf.name))
        skipcount += 1
        break

      # print(file_path)
      page.save(file_path)
    fcount += 1
    # debugging
    # if fcount > 0:
    #   break

pdf_to_images(os.path.join(BASE_PATH,DATA_PATH,'CHI-2019-pdfs-RETITLED'),os.path.join(BASE_PATH,DATA_PATH,'CHI-2019-pngs'))

938 PDFs ready to be converted
0: Converting MicrosoftWordpaper493docx.pdf
MicrosoftWordpaper493docx.pdf already converted. Skipping.
1: Converting AutomatingtheAdministrationandAnalysisofPsychiatricTestsTheCaseofAttachmentinSchoolAgeChildren.pdf
2: Converting AggregatedVisualizationofPlaytestingData.pdf
3: Converting FeelingFireworksAnInclusiveTactileFireworkDisplay.pdf
4: Converting ModelingMobileInterfaceTappabilityUsingCrowdsourcingandDeepLearning.pdf
5: Converting InsertYourTitleHere.pdf
6: Converting CrossDeviceTaxonomySurveyOpportunitiesandChallengesofInteractionsSpanningAcrossMultipleDevices.pdf
7: Converting PseudoHapticWeightChangingthePerceivedWeightofVirtualObjectsByManipulatingControlDisplayRatio.pdf
8: Converting DiagnosingandCopingwithModeErrorsinKoreanEnglishDuallanguageKeyboard.pdf
9: Converting TowardsAugmentingIVRCommunicationwithPhysiologicalSensingData.pdf
10: Converting MicrosoftWorddc1064JillianLWarrenCHI2019camerareadydocx.pdf
11: Converting ORCLayoutAdaptiveGUI

#Clean up

In [ ]:
# Delete PDFs and leave only .zip and image dir
!pwd
!ls
!rm -rf CHI-2019-pdfs-RETITLED

/gdrive/Shareddrives/deep learning and PDF accessibility/dataset/CHI-2019
CHI-2019-pdfs-RETITLED	CHI-2019-pdfs-RETITLED.zip  CHI-2019-pngs  __MACOSX


# Compare with HTMLs

In [ ]:
# note if we use lower() we get more matches.
# need to look into this
import collections
os.chdir(os.path.join(BASE_PATH,DATA_PATH))
html_dir = os.path.join(BASE_PATH,DATA_PATH,'CHI-2019-htmls-incomplete')
htmls = pathlib.Path(html_dir).rglob('*.htm*')
html_stems = [html.stem for html in htmls]

img_dir = os.path.join(BASE_PATH,DATA_PATH,'CHI-2019-jpegs-overkill')
imgs = pathlib.Path(img_dir).rglob('*.jpeg')
img_stems = [img.stem.split('-')[0] for img in imgs]
page_counter = collections.Counter(img_stems)

matches = set(img_stems).intersection(html_stems)
print('We have {} paper title matches between the images and htmls.'.format(len(matches)))

total_pages = 0
for match in matches:
  total_pages += page_counter[match]
print('This is a total of {} pages.'.format(total_pages))

We have 465 paper title matches between the images and htmls.
This is a total of 5841 pages.


In [ ]:
import shutil
# now i want to move all the jpegs that are matches into their own directory
# have to recreate the generator because I don't know how to rewind one
imgs = pathlib.Path(img_dir).rglob('*.jpeg')
dst_dir = os.path.join(BASE_PATH,DATA_PATH,'CHI-2019-jpegs')

match_count = 0
for img in imgs:
  stem = img.stem.split('-')[0]
  if stem in matches:
    img_copy = os.path.join(dst_dir,img.name)
    shutil.copyfile(img,img_copy)
    match_count += 1

print('Copied {} files.'.format(match_count))

Copied 5841 files.
